In [1]:
import os
import nibabel as nib
import numpy as np
from scipy import ndimage
import glob

In [2]:
class nii_process:
    def __init__(self, base_):
        self.base_ = base_
        self.volume = np.array([])
    
    def slice_normalize(self, volume):
        # print(volume.shape)
        img_o = np.float32(volume.copy())
        for i in range(self.slice_n):
            img_o = np.float32(volume[...,i].copy())
            m = np.mean(img_o)
            s = np.std(img_o)
            volume[...,i] = np.divide((img_o - m), s)
        print(np.max(volume[...,i]), np.min(volume[...,i]))
        # volume = np.divide(volume, np.percentile(volume,98))
        self.image = volume.astype('float32')

    def normalize(self, volume):
        img_o = np.float32(volume.copy())
        m = np.mean(img_o)
        s = np.std(img_o)
        volume = np.divide((img_o - m), s)
        image = volume.astype("float32")
        return image
    
    def process_scan(self, dstype, imgtype,path):

        image_o = nib.load(path)
        affine = image_o.header.get_best_affine()

        if len(image_o.shape) == 4:
            image = image_o.get_fdata()
            width,height,queue,_ = image.shape
            image = image[:,:,:,1]
            image = np.reshape(image,(width,height,queue))
            adjusted_dwi = nib.Nifti1Image(image.astype(np.uint16), affine)
        else:
            image = image_o.get_fdata()
            pass
        self.slice_n = image.shape[-1]
        if imgtype=='image':
            image = self.normalize(image)
        nii_name_slices = (os.path.split(path))
        nii_name_slices = nii_name_slices[1].split('.')[0]
        # print(nii_name_slices)
        # print(image_o.shape)
        for i in range(self.slice_n):
            # print(image[...,i].shape)
            adjusted_seg = nib.Nifti1Image(image[...,i], affine)
            adjusted_seg.header['pixdim'] = image_o.header['pixdim']
            # Save as NiBabel file
            adjusted_seg.to_filename(os.path.join(self.base_, dstype, imgtype,f'{nii_name_slices[0:-1]}_{i+1}.nii.gz'))

In [3]:
# all nii image transfer single slice image (20,384,384) - > 20,(384,384)
prepare_data = nii_process('./dataset/normalized_zscore/')
process_stack = ['train','valid', 'test']
for i in process_stack:
    _list = sorted(os.listdir(os.path.join('./dataset/original_data/', i)))
    _len = len(_list)
    print(_len)
    for j in range(0, _len, 2):
        prepare_data.process_scan(i, 'image',f'./dataset/original_data/{i}/{_list[j]}')
        prepare_data.process_scan(i, 'masks',f'./dataset/original_data/{i}/{_list[j+1]}')

310
72
90


In [4]:
# from intensity_normalization.normalize.fcm import FCMNormalize
# from intensity_normalization.normalize.zscore import ZScoreNormalize
# from tqdm import tqdm
# path_in = "./dataset/original_data/"
# save_path_ = './dataset/rawdata_fcm/'
# process_stack = ['train','valid']
# for i in process_stack:
#     count=0
#     _list = sorted(os.listdir(os.path.join(path_in, i)))
#     _len = len(_list)
#     for j in tqdm(range(0, _len, 2)):
#         count+=1
#         image = nib.load(os.path.join(path_in, i, _list[j]))  # assume skull-stripped otherwise load mask too
#         fcm_norm = FCMNormalize()
#         normalized = fcm_norm(image) # alternatively, you can pass in a numpy array which will return a numpy array
#         normalized.to_filename(os.path.join(save_path_, i, _list[j]))  # this works if you passed in a nibabel Nifti image
#     print(i, count)